In [ ]:
%load_ext autoreload
%autoreload 2


%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12

import covid19

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
!git -C ../../covid19-opendata-vaccini pull

In [ ]:
consegne = pd.read_csv('../../covid19-opendata-vaccini/dati/consegne-vaccini-latest.csv')
consegne

In [ ]:
path = '../../covid19-opendata-vaccini/dati/somministrazioni-vaccini-latest.csv'
vaccines_italy = covid19.data.read_vaccini(path)
vaccines_italy

In [ ]:
doses = vaccines_italy.dose1 + vaccines_italy.dose2
doses = doses.loc[{"provider": slice(None, None, -1)}]
doses0 = doses.sum("age_class")
doses70 = doses.sel(age_class=slice("70-79", None)).sum("age_class")
doses80 = doses.sel(age_class=slice("80-89", None)).sum("age_class")

doses0_sum = doses0.cumsum("time")
doses70_sum = doses70.cumsum("time")
doses80_sum = doses80.cumsum("time")

In [ ]:
doses0_pm = doses0 / doses70.population * 1_000_000
doses70_pm = doses70 / doses70.population_70 * 1_000_000
doses80_pm = doses80 / doses80.population_80 * 1_000_000

doses0_sum_pm = doses0_pm.cumsum("time")
doses70_sum_pm = doses70_pm.cumsum("time")
doses80_sum_pm = doses80_pm.cumsum("time")

In [ ]:
location = "Italia"
ax = covid19.plot.stack_xarray(doses0_sum_pm.sel(location=location) / 1_000_000 / 2, hue="provider", window=7, title=f"Popolazione - {location}", ylim=(0, 1))
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% dosi somministrate")
ax.legend() 

In [ ]:
location = "Italia"
ax = covid19.plot.stack_xarray(doses70_sum_pm.sel(location=location) / 1_000_000 / 2, hue="provider", window=7, title=f"Popolazione over 70 - {location}", ylim=(0, 1))
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% dosi somministrate")
ax.legend() 

In [ ]:
location = "Italia"
ax = covid19.plot.stack_xarray(doses80_sum_pm.sel(location=location) / 1_000_000 / 2, hue="provider", window=7, title=f"Popolazione over 80 - {location}", ylim=(0, 1))
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% dosi somministrate")
ax.legend() 

In [ ]:
doses80_today = doses80_sum.sum("provider")
doses80_todo = doses80_sum.population_80 * 2 - doses80_today
doses80_last_week =  doses80_sum.shift({"time": 2}).sum("provider") - doses80_sum.shift({"time": 9}).sum("provider")
doses80_weeks_todo = doses80_todo / doses80_last_week
doses80_today_perc = doses80_today / (2 * doses80_todo.population_80)
doses80_alldone =  doses80_weeks_todo * 7 * pd.Timedelta(1, "D") + doses80_today.time
doses80_alldone

In [ ]:
ax = covid19.plot.scatter_xarray(doses80_alldone.isel(time=slice(-7, None)), doses80_today_perc, xlim=(np.datetime64('2021-03-01'), np.datetime64('2022-03-01')), ylim=(0,1))
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
_ = ax.set(title="Popolazione over 80 - Italia", ylabel="% dosi somministrate", xlabel="data stimata per complere le vaccinazioni")

In [ ]:
doses70_today = doses70_sum.sum("provider")
doses70_todo = doses70_sum.population_70 * 2 - doses80_today
doses70_last_week =  doses70_sum.shift({"time": 2}).sum("provider") - doses70_sum.shift({"time": 9}).sum("provider")
doses70_weeks_todo = doses70_todo / doses70_last_week
doses70_today_perc = doses70_today / (2 * doses70_todo.population_70)
doses70_alldone =  doses70_weeks_todo * 7 * pd.Timedelta(1, "D") + doses70_today.time
doses70_alldone

In [ ]:
ax = covid19.plot.scatter_xarray(doses70_alldone.isel(time=slice(-7, None)), doses80_today_perc, xlim=(np.datetime64('2021-06-01'), np.datetime64('2022-06-01')), ylim=(0,1))
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
_ = ax.set(title="Popolazione over 70 - Italia", ylabel="% dosi somministrate", xlabel="data stimata per complere le vaccinazioni")

In [ ]:
for location in doses80_sum.location.values:
    ax = covid19.plot.stack_xarray(doses80_sum_pm.sel(location=location) / 1_000_000 / 2, hue="provider", window=7, title=f"Popolazione over 80 - {location}", ylim=(0, 1))
    ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
    ax.set(ylabel="% dosi somministrate")
    ax.legend()

In [ ]:
for location in doses0_sum.location.values:
    ax = covid19.plot.stack_xarray(doses0_sum_pm.sel(location=location) / 1_000_000 / 2, hue="provider", window=7, title=f"Popolazione - {location}", ylim=(0, 1))
    ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
    ax.set(ylabel="% dosi somministrate")
    ax.legend()